In [11]:
import numpy as np
import h5py
import sys
import matplotlib.pyplot as pl
from stringato import extract_floats as ef
import numpy as np
from scipy.spatial.distance import pdist
from numba  import jit
import os
from scipy.interpolate import UnivariateSpline
from mayavi import mlab
import ipyvolume as ipv

def reassign(b,shape):
    dummy = np.zeros(shape)
    s0 = shape[0]
    for s in range(shape[2]):
        dummy[:,:,s] = b[s*s0:(s+1)*s0,:]
    return dummy

Pe =60.0
L = 5
N = 100
rmin=1.0
potential = np.loadtxt(f"../Interactions/threebodypotential-L5-Pe{Pe}.txt")

depletion = np.loadtxt(f"../Interactions/depletion-Pe{Pe}L4.0.txt")

tb = UnivariateSpline(depletion[:,0], depletion[:,1],s=0)

def twobody(r):
	return np.piecewise(r,[r<1.0, r>2.5, ],[0,0,tb])

# rr = np.linspace(0.9,2.5)
# pl.plot(rr,twobody(rr))
# pl.show()
# exit()

def twobody_triangle(r1,r2,r3):
	U= twobody(r1)+twobody(r2)+twobody(r3)

	forbidden = (r1>r2)+(r3<r2)+(r3<r1)+(r1<rmin)+(r2<rmin)+(r3<rmin)+(r1>2.5)+(r2>2.5)+(r3>2.5)
	U[forbidden]=0
	return U

def sticky_triangle(r1,r2,r3, epsilon,):
	U= (r1<2.5)*(r2<2.5)*(r3<2.5)*epsilon
	forbidden = (r1>r2)+(r3<r2)+(r3<r1)+(r1<rmin)+(r2<rmin)+(r3<rmin)+(r1>2.5)+(r2>2.5)+(r3>2.5)
	U[forbidden]=0
	return U

potential = reassign(potential,[N,N,N])
bins = np.linspace(0, L, N+1)
cnt = bins[:-1]+(bins[1]-bins[0])/2

R1,R2,R3 = np.meshgrid(cnt,cnt,cnt, indexing="ij")

twopotential =  twobody_triangle(R2,R1,R3)

# potential -= twopotential
# potential = sticky_triangle(R2,R1,R3,-0.5)
ipv.quickvolshow(potential, level=[-0.5, 0.0], opacity=0.03, level_width=0.1, data_min=potential.min(), data_max=potential.max())

# print(potential.max(), potential.min())
# # potential[potential>0]=0R2,R1,R3
# with open(f"../Interactions/threebodypotential-minustwo-L5-Pe{Pe}.txt", 'w') as fout:
#                 for s in range(potential.shape[2]):
#                     np.savetxt(fout,potential[:,:,s])



